In [2]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Dense, Dropout, Input, Conv2D, MaxPooling2D, Flatten
from keras.models import Model

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10287249359276943439
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7076629709
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12935333050259504116
physical_device_desc: "device: 0, name: GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [4]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [5]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [7]:
# load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#get number of labels
num_labels = len(np.unique(y_train))

In [8]:
num_labels

10

In [9]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [11]:
image_size = x_train.shape[1]
image_size

28

In [12]:
# reshape and normalize input images
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [13]:
# network parameters
input_shape = (image_size, image_size, 1)
batch_size = 128
kernel_size = 3
filters = 64
dropout = 0.3

In [14]:
# use functional API to build a model
inputs = Input(shape=input_shape)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(inputs)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
# image to vector before connecting to dense layer
y = Flatten()(y)
# dropout
y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)

# build the model 
model = Model(inputs=inputs, outputs=outputs)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
__________

In [15]:
# classifier loss, Adam optimizer, classifier accuracy
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
# training
model.fit(x_train,
          y_train,
          validation_data=(x_test, y_test),
          epochs=20,
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 6s 97us/step - loss: 0.2774 - acc: 0.9147 - val_loss: 0.0549 - val_acc: 0.9815
Epoch 2/20
60000/60000 [==============================] - 3s 53us/step - loss: 0.0724 - acc: 0.9777 - val_loss: 0.0389 - val_acc: 0.9862
Epoch 3/20
60000/60000 [==============================] - 3s 56us/step - loss: 0.0515 - acc: 0.9840 - val_loss: 0.0287 - val_acc: 0.9899
Epoch 4/20
60000/60000 [==============================] - 3s 56us/step - loss: 0.0422 - acc: 0.9868 - val_loss: 0.0262 - val_acc: 0.9901
Epoch 5/20
60000/60000 [==============================] - 3s 55us/step - loss: 0.0349 - acc: 0.9892 - val_loss: 0.0275 - val_acc: 0.9903
Epoch 6/20
60000/60000 [==============================] - 3s 54us/step - loss: 0.0309 - acc: 0.9903 - val_loss: 0.0240 - val_acc: 0.9915
Epoch 7/20
60000/60000 [==============================] - 3s 58us/step - loss: 0.0273 - acc: 0.9910 - val_loss: 0.0252 - val_acc

In [18]:
# Evaluate on test dataset
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.2f%%" % (100.0 * score[1]))

10000/10000 [==============================] - 0s 22us/step

Test accuracy: 99.32%
